# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
#Your code here; create the connection and cursor
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [4]:
#Your code here; use a subquery. No join will be necessary.

c.execute('''select customerNumber, contactLastName, contactFirstName 
                    from customers where customerNumber in
                    (select customerNumber from orders where orderDate = '2003-01-31')''').fetchall()

[('141', 'Freyre', 'Diego ')]

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [18]:
#Your code here

c.execute('''select productName, quantityOrdered 
                    from products join orderdetails using(productCode) group by 1 order by 2 DESC''').fetchall()

# c.execute('''select productName, quantityOrdered from products
#                     where quantityOrdered in (select quantityOrdered from orderdetails) group by 1''')

[('1969 Dodge Charger', '97'),
 ('1969 Dodge Super Bee', '90'),
 ('America West Airlines B757-200', '85'),
 ('1970 Plymouth Hemi Cuda', '77'),
 ('F/A 18 Hornet 1/72', '77'),
 ('1917 Grand Touring Sedan', '76'),
 ('1949 Jaguar XK 120', '76'),
 ('1956 Porsche 356A Coupe', '76'),
 ('1962 Volkswagen Microbus', '70'),
 ('1932 Alfa Romeo 8C2300 Spider Sport', '66'),
 ('1957 Ford Thunderbird', '66'),
 ('1969 Harley Davidson Ultimate Chopper', '66'),
 ('2002 Suzuki XREO', '66'),
 ('2003 Harley-Davidson Eagle Drag Bike', '66'),
 ('1937 Lincoln Berline', '65'),
 ('1948 Porsche Type 356 Roadster', '65'),
 ('1966 Shelby Cobra 427 S/C', '64'),
 ('1968 Ford Mustang', '64'),
 ('1972 Alfa Romeo GTA', '64'),
 ('Corsair F4U ( Bird Cage)', '64'),
 ('1928 Ford Phaeton Deluxe', '62'),
 ('1939 Chevrolet Deluxe Coupe', '61'),
 ('1948 Porsche 356-A Roadster', '61'),
 ('1993 Mazda RX-7', '61'),
 ('1912 Ford Model T Delivery Wagon', '60'),
 ('1970 Chevy Chevelle SS 454', '60'),
 ('1992 Ferrari 360 Spider red', 

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [23]:
#Your code here
c.execute('''select productName, sum(customerNumber) from products
                    join orderdetails using(productCode)
                    join orders using(orderNumber) group by 1''').fetchall()

[('18th Century Vintage Horse Carriage', 8061),
 ('18th century schooner', 6813),
 ('1900s Vintage Bi-Plane', 9383),
 ('1900s Vintage Tri-Plane', 7103),
 ('1903 Ford Model A', 6524),
 ('1904 Buick Runabout', 6464),
 ('1911 Ford Town Car', 7176),
 ('1912 Ford Model T Delivery Wagon', 7138),
 ('1913 Ford Model T Speedster', 7897),
 ('1917 Grand Touring Sedan', 7192),
 ('1917 Maxwell Touring Car', 7548),
 ('1926 Ford Fire Engine', 6512),
 ('1928 British Royal Navy Airplane', 7758),
 ('1928 Ford Phaeton Deluxe', 7645),
 ('1928 Mercedes-Benz SSK', 6560),
 ('1930 Buick Marquette Phaeton', 7742),
 ('1932 Alfa Romeo 8C2300 Spider Sport', 6730),
 ('1932 Model A Ford J-Coupe', 7452),
 ('1934 Ford V8 Coupe', 8161),
 ('1936 Chrysler Airflow', 8045),
 ('1936 Harley Davidson El Knucklehead', 6750),
 ('1936 Mercedes Benz 500k Roadster', 7052),
 ('1936 Mercedes-Benz 500K Special Roadster', 7391),
 ('1937 Horch 930V Limousine', 9169),
 ('1937 Lincoln Berline', 6865),
 ('1938 Cadillac V-16 Presidential 

## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
#Your code here

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!